In [1]:
# Read fits files
import fitsio
import numpy as np

pym_name = 'manga-pymorph-DR15.fits'
mor_name = 'manga-morphology-dl-DR15.fits'
g = fitsio.read(pym_name,1)
r = fitsio.read(pym_name,2)
mor = fitsio.read(mor_name,1)

In [2]:
print(g.dtype.names)
print(r.dtype.names)
print(mor.dtype.names)
print(r[0])

('INTID', 'MANGA_ID', 'PLATEIFU', 'OBJID', 'RA', 'DEC', 'Z', 'EXTINCTION', 'DUPL_GR', 'DUPL_N', 'DUPL_ID', 'FLAG_FIT', 'FLAG_FAILED_S', 'M_S', 'M_S_ERR', 'M_S_TRUNC', 'A_HL_S', 'A_HL_S_ERR', 'A_HL_S_TRUNC', 'N_S', 'N_S_ERR', 'BA_S', 'BA_S_ERR', 'PA_S', 'PA_S_ERR', 'GALSKY_S', 'FLAG_FAILED_SE', 'M_SE', 'M_SE_TRUNC', 'A_HL_SE', 'A_HL_SE_TRUNC', 'BA_SE', 'BT_SE', 'BT_SE_TRUNC', 'M_SE_BULGE', 'M_SE_BULGE_ERR', 'M_SE_BULGE_TRUNC', 'A_HL_SE_BULGE', 'A_HL_SE_BULGE_ERR', 'A_HL_SE_BULGE_TRUNC', 'N_SE_BULGE', 'N_SE_BULGE_ERR', 'BA_SE_BULGE', 'BA_SE_BULGE_ERR', 'PA_SE_BULGE', 'PA_SE_BULGE_ERR', 'M_SE_DISK', 'M_SE_DISK_ERR', 'M_SE_DISK_TRUNC', 'A_HL_SE_DISK', 'A_HL_SE_DISK_ERR', 'A_HL_SE_DISK_TRUNC', 'N_SE_DISK', 'N_SE_DISK_ERR', 'BA_SE_DISK', 'BA_SE_DISK_ERR', 'PA_SE_DISK', 'PA_SE_DISK_ERR', 'GALSKY_SE')
('INTID', 'MANGA_ID', 'PLATEIFU', 'OBJID', 'RA', 'DEC', 'Z', 'EXTINCTION', 'DUPL_GR', 'DUPL_N', 'DUPL_ID', 'FLAG_FIT', 'FLAG_FAILED_S', 'M_S', 'M_S_ERR', 'M_S_TRUNC', 'A_HL_S', 'A_HL_S_ERR', 'A_H

In [3]:
# Split full data set into training and validation halves
ntot = len(r)
ntrain = ntot // 2
rng = np.random.RandomState(314159)
selection = rng.choice(ntot, size=ntrain, replace=False)
print(selection)
train = np.in1d(range(ntot),selection)
valid = ~train
print('m1 =',train)
print('m2 =',valid)
print('ntot = ',ntot)
print('ntrain = ',np.sum(train))
print('nvalidate = ',np.sum(valid))

[4214 3666 1693 ... 2250 4187 3571]
m1 = [ True  True False ... False False  True]
m2 = [False False  True ...  True  True False]
ntot =  4672
ntrain =  2336
nvalidate =  2336


In [4]:
# Not all rows have valid fits.  Define a mask to exclude bad S/E fits
flag_fit = r['FLAG_FIT']
print(np.unique(flag_fit))
print('N 0 =',np.sum(flag_fit==0))
print('N 1 =',np.sum(flag_fit==1))
print('N 2 =',np.sum(flag_fit==2))
print('N 3 =',np.sum(flag_fit==3))
r_failed_se = r['FLAG_FAILED_SE']
r_failed_s = r['FLAG_FAILED_S']
g_failed_se = g['FLAG_FAILED_SE']
g_failed_s = g['FLAG_FAILED_S']
print('N failed se =',np.sum(r_failed_se))
print('N failed s =',np.sum(r_failed_s))
print('N tot = ',len(r))
mask = (r_failed_se == 0) & (g_failed_se == 0)
print('N good =',len(r[mask]))
print('N good with flag_fit=3 =',np.sum(flag_fit[mask]==3))

[0 1 2 3]
N 0 = 411
N 1 = 2367
N 2 = 1696
N 3 = 198
N failed se = 406
N failed s = 333
N tot =  4672
N good = 4126
N good with flag_fit=3 = 0


In [5]:
# Define types from TTYPE and P_S0 as per Helena's instructions.
Ell = (mor['TTYPE']<=0.) & (mor['P_S0']<=0.5)
S0 = (mor['TTYPE']<=0.) & (mor['P_S0']>0.5)
S1 = (mor['TTYPE']>0.) & (mor['TTYPE']<=3.)
S2 = mor['TTYPE']>3.
print(np.sum(Ell))
print(np.sum(S0))
print(np.sum(S1))
print(np.sum(S2))
# Make sure nothing is 2 types
assert np.sum(Ell & S0) == 0
assert np.sum(Ell & S1) == 0
assert np.sum(Ell & S2) == 0
assert np.sum(S0 & S1) == 0
assert np.sum(S0 & S2) == 0
assert np.sum(S1 & S2) == 0
# Make sure everything is some type
print(len(mor))
assert np.sum(Ell | S0 | S1 | S2) == len(mor)
gal_type = np.zeros(len(mor), dtype=int)
gal_type[Ell] = 1
gal_type[S0] = 2
gal_type[S1] = 3
gal_type[S2] = 4

1052
896
902
1822
4672


In [6]:
# Get some easy columns
id = r['INTID']
ra = r['RA']
dec = r['DEC']
z = r['Z']
re = r['A_HL_SE']
pa = r['PA_SE_DISK']  # No overall PA, but the disk one is probably sufficient for this.
b_over_a = r['BA_SE']
bulge_frac = r['BT_SE']
n_bulge = r['N_SE_BULGE']
re_bulge = r['A_HL_SE_BULGE']
re_disk = r['A_HL_SE_DISK']

In [7]:
# Convert distances to kpc (currently arcsec)
print('max z =',np.max(z))
print('z =',z)
dA = z/(1+z) * (3.e5) / 70 * 1.e6  # pc
print('dA (pc) =',dA[mask])
kpc = (1/206265.) * dA * 1.e-3  # kpc / arcsec
print('kpc = ',kpc[mask])
print('re (arcsec) =',re[mask])
print('re (kpc) =',(re * kpc)[mask])
re *= kpc
re_bulge *= kpc
re_disk *= kpc

max z = 0.14951847955617875
z = [0.04288711 0.04496038 0.05276287 ... 0.0691559  0.0227374  0.0550323 ]
dA (pc) = [1.84396775e+08 2.14793474e+08 1.05982310e+08 ... 4.38523686e+07
 4.94947172e+07 2.77211616e+08]
kpc =  [0.89397996 1.04134717 0.51381625 ... 0.21260208 0.23995693 1.34395858]
re (arcsec) = [ 7.15706086  5.36164366  8.01950677 ...  4.64494896 19.88123894
  5.1775279 ]
re (kpc) = [6.39826895 5.58333244 4.12055292 ... 0.98752582 4.77064116 6.95838305]


In [8]:
# Figure out the Luminosity and Color
rmag = r['M_SE'] # apparent magnitude
gmag = g['M_SE'] # apparent magnitude
print('r =',rmag[mask])
print('g =',gmag[mask])
dL = dA * (1+z)**2
rabs = rmag - 5 * np.log10(dL / 10)
gabs = gmag - 5 * np.log10(dL / 10)
print('rabs = ',rabs[mask])
print('gabs = ',gabs[mask])
mabs_sun = 4.83  # ignore k correction...
Lr = np.zeros(ntot)
Lg = np.zeros(ntot)
C = np.zeros(ntot)
Lr[mask] = 10**(0.4 * (mabs_sun - rabs[mask]))
Lg[mask] = 10**(0.4 * (mabs_sun - gabs[mask]))
C[mask] = Lg[mask]/Lr[mask]
print('Lr (Lsun) = ',Lg[mask])
print('Lr (Lsun) = ',Lr[mask])
print('C = Lg/Lr = ',C[mask])

r = [16.38410785 15.63347435 14.30344009 ... 15.5816927  12.96950817
 16.22357178]
g = [17.04116821 16.35520172 14.88530159 ... 15.98082924 13.73615646
 16.94196892]
rabs =  [-20.13565699 -21.24993666 -20.93147461 ... -17.67293946 -20.55373359
 -21.28089572]
gabs =  [-19.47859662 -20.52820929 -20.34961312 ... -17.27380292 -19.7870853
 -20.56249858]
Lr (Lsun) =  [5.28979266e+09 1.39086096e+10 1.17990012e+10 ... 6.94258438e+08
 7.02803830e+09 1.43548756e+10]
Lr (Lsun) =  [9.68863946e+09 2.70380062e+10 2.01646108e+10 ... 1.00271101e+09
 1.42394573e+10 2.78200746e+10]
C = Lg/Lr =  [0.54597889 0.51440958 0.58513409 ... 0.69238138 0.49356083 0.51598983]


In [9]:
# Make ASCII catalogs:
fname1 = 'training_galaxies.dat'
fname2 = 'validation_galaxies.dat'

for fname, m in ((fname1, train), (fname2, valid)):
    print(fname,'\n')
    with open(fname,'w+') as f:
        f.write('#  id  gal_type   ra          dec       redshift  log_luminosity  color      radius     b_over_a   pos_angle   bulge_fract   sersic_n    r_bulge     r_disk\n')
        for i in range(len(id)):
            if not m[i]: continue
            if not mask[i]: continue
            f.write(("%5d   %3d " + "  %10.5f" * 12 + "\n")%(id[i], gal_type[i],
                                                            ra[i], dec[i], z[i], np.log10(Lr[i]), C[i],
                                                            re[i], b_over_a[i], pa[i], bulge_frac[i],
                                                            n_bulge[i], re_bulge[i], re_disk[i]))
    with open(fname,'r') as f:
        lines = f.read().split('\n')
        print('\n'.join(lines[:10]))
    print()

training_galaxies.dat 

#  id  gal_type   ra          dec       redshift  log_luminosity  color      radius     b_over_a   pos_angle   bulge_fract   sersic_n    r_bulge     r_disk
    2     4    157.05886    42.96805     0.04496     9.98626     0.54598     6.39827     0.17906    57.42190     0.06485     1.30600     5.81586     6.37754
    6     4    129.89352    25.47709     0.02918     9.96684     0.53171     4.86181     0.93007   -29.24130     0.11893     1.00000     1.09123     5.33112
    7     4    192.88862    26.76983     0.04561    10.22729     0.68002     7.24744     0.61682   -48.55630     0.11306     1.54460     1.58634     8.18503
    9     2    257.95754    57.11093     0.03028    10.40243     0.48439     3.94840     0.51641   -43.54110     0.56902     2.72450     3.06734     5.38915
   16     1    182.35628    46.54936     0.06909    10.82827     0.47607     9.32379     0.93203    49.65370     0.85496     7.17250    10.18022     7.76279
   17     1    117.39779    30.4166

In [10]:
# Check that numpy reads it back in correctly
fname1 = 'training_galaxies.dat'
data = np.genfromtxt(fname1, names=True, dtype=None)
print(data)
print(data.dtype.names)
print(len(data))

[(   2, 4, 157.05886, 42.96805, 0.04496,  9.98626, 0.54598, 6.39827, 0.17906,  57.4219, 0.06485, 1.306 , 5.81586, 6.37754)
 (   6, 4, 129.89352, 25.47709, 0.02918,  9.96684, 0.53171, 4.86181, 0.93007, -29.2413, 0.11893, 1.    , 1.09123, 5.33112)
 (   7, 4, 192.88862, 26.76983, 0.04561, 10.22729, 0.68002, 7.24744, 0.61682, -48.5563, 0.11306, 1.5446, 1.58634, 8.18503)
 ...
 (4696, 4, 165.10414, 43.01969, 0.03743,  9.88758, 0.95614, 1.7786 , 0.50697,  23.8471, 0.62913, 3.4918, 1.9188 , 1.37429)
 (4698, 4, 149.90212, 34.32022, 0.02089,  9.494  , 0.74093, 2.98093, 0.74259, -71.3918, 0.20503, 1.    , 0.95447, 3.66181)
 (4702, 4, 113.8983 , 41.96359, 0.01034,  9.00118, 0.69238, 0.98753, 0.68095, -21.999 , 0.32864, 0.4   , 1.04857, 1.01929)]
('id', 'gal_type', 'ra', 'dec', 'redshift', 'log_luminosity', 'color', 'radius', 'b_over_a', 'pos_angle', 'bulge_fract', 'sersic_n', 'r_bulge', 'r_disk')
2063


In [11]:
# Write the full, original data into ASCII files
names = list(g.dtype.names)
print(names)

for band, data in (('g', g), ('r', r)):
    fname1 = 'training_galaxies_full_%s.dat'%band
    fname2 = 'validation_galaxies_full_%s.dat'%band

    for fname, m in ((fname1, train), (fname2, valid)):
        print(fname,'\n')
        with open(fname,'w+') as f:
            f.write('# ID ' + '  '.join(names[3:]) + '\n')
            for i in range(len(id)):
                if not m[i]: continue
                if not mask[i]: continue
                values = [id[i]] + list(data[i])[3:]
                str_values = [str(v) for v in values]
                f.write('  '.join(str_values) + '\n')
                
data = np.genfromtxt(fname1, names=True, dtype=None, encoding=None)
print(data)
print(data.dtype.names)

['INTID', 'MANGA_ID', 'PLATEIFU', 'OBJID', 'RA', 'DEC', 'Z', 'EXTINCTION', 'DUPL_GR', 'DUPL_N', 'DUPL_ID', 'FLAG_FIT', 'FLAG_FAILED_S', 'M_S', 'M_S_ERR', 'M_S_TRUNC', 'A_HL_S', 'A_HL_S_ERR', 'A_HL_S_TRUNC', 'N_S', 'N_S_ERR', 'BA_S', 'BA_S_ERR', 'PA_S', 'PA_S_ERR', 'GALSKY_S', 'FLAG_FAILED_SE', 'M_SE', 'M_SE_TRUNC', 'A_HL_SE', 'A_HL_SE_TRUNC', 'BA_SE', 'BT_SE', 'BT_SE_TRUNC', 'M_SE_BULGE', 'M_SE_BULGE_ERR', 'M_SE_BULGE_TRUNC', 'A_HL_SE_BULGE', 'A_HL_SE_BULGE_ERR', 'A_HL_SE_BULGE_TRUNC', 'N_SE_BULGE', 'N_SE_BULGE_ERR', 'BA_SE_BULGE', 'BA_SE_BULGE_ERR', 'PA_SE_BULGE', 'PA_SE_BULGE_ERR', 'M_SE_DISK', 'M_SE_DISK_ERR', 'M_SE_DISK_TRUNC', 'A_HL_SE_DISK', 'A_HL_SE_DISK_ERR', 'A_HL_SE_DISK_TRUNC', 'N_SE_DISK', 'N_SE_DISK_ERR', 'BA_SE_DISK', 'BA_SE_DISK_ERR', 'PA_SE_DISK', 'PA_SE_DISK_ERR', 'GALSKY_SE']
training_galaxies_full_g.dat 

validation_galaxies_full_g.dat 

training_galaxies_full_r.dat 

validation_galaxies_full_r.dat 

[(   2, 1237660634384761005, 157.05886   , 42.96805   , 0.04496038,